# Scrape and Save
This notebook was written to automate the scraping and saving of images from the internet.

Import the relevant modules.

In [10]:
pip install webdriver-manager

Note: you may need to restart the kernel to use updated packages.


In [11]:
import requests
from bs4 import BeautifulSoup
import os
from selenium import webdriver 
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager


The next step is to get the html as text from the url.

In [12]:
#chromePath='C:\\Users\\Lenovo\Downloads\\chromedriver_win32'
#driver=webdriver.Chrome(chromePath) 

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

url = "https://www.thecarconnection.com/overview/acura_ilx_2022"
driver.get(url)

[WDM] - Downloading: 100%|██████████| 6.58M/6.58M [00:00<00:00, 8.33MB/s]


Next, we use BeautifulSoup to parse the html text data for the image links and then save them in a list.

In [13]:
driver.execute_script("window.scrollTo(0, 0);")

html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')
containers = soup.findAll('div', {'class':"isv-r PNCib MSM1fd BUooTd"} )

os is used to a make the folder into which the files will be saved. A list of directory names corresponding to the category of images should be created.

In [14]:
os.getcwd()

'c:\\Users\\Lenovo\\image_saver'

In [3]:
dirs = ['C:\\Users\\Lenovo\\Images']

Finally, iterating through the list of image urls, download and save the images in the appropriate directory. 

In [16]:
image_elements = soup.find_all('img')

In [17]:
img_urls = []
for element in image_elements:
     img_urls.append(element['src'])  

In [18]:
for i, url in enumerate(img_urls):
  try: response = requests.get(url)
  except: pass
  
  if response.status_code:
    with open('{0}/test_save_{1}.png'.format(dirs[0], i), 'wb') as fp:
          fp.write(response.content)

# Saving to Google Cloud

1. Install gcloud https://cloud.google.com/sdk/docs/install#installation_instructions. 

2. Follow the instructions here https://cloud.google.com/docs/authentication/provide-credentials-adc#local-dev. 

3. Follow the instructions at https://cloud.google.com/docs/authentication/client-libraries to set up Google API credentials.

In [1]:
from google.cloud import storage
import glob

project_id = 'vehicle-recognition-model'

storage_client = storage.Client(project=project_id)

bucket_name = 'vehicle-recognition-model'

bucket = storage_client.get_bucket(bucket_name)

In [4]:
for str_path_file in glob.glob(dirs[0] + '/**'):
    blob = bucket.blob(str_path_file)
    blob.upload_from_filename(str_path_file)

We'll need to change this script in the following ways


* It will need to be able to download text from an accessible website or google images; https://www.thecarconnection.com/ is not accessible via this method.
* Depending on the website, the script may need to be able to perform searches 
* We will need to direct it to save the images in a Google Cloud bucket directory.
* Filter out unneccsary images
* How much is too much to download at a time
* Upload directly to cloud bucket
